# Smart signatures
#### 06.1 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2023-02-10 (started 2022-01-12)

* Write and deploy smart Signatures

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [ ]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [ ]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk import transaction
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import hashlib

In [ ]:
from pyteal import *

In [ ]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])
algod_client.status()["last-round"]

## The Escrow
A classical smart contact. **Alice** pays to **Bob** (and only to **Bob**).

**Bob** can decide when to ask for the money.

##### Step 1: The programmer writes down the conditions as a PyTeal program

In [ ]:
escrow_pyteal =  (
    Txn.receiver() == Addr(Bob["public"])            # Receipient must be Bob
)                                                    # Encode addresses using Addr()
                                                     # No random number here (why?)

# Security missing ... do not copy-paste

##### Step 2: Compile PyTeal -> Teal

In [ ]:
escrow_teal = compileTeal(escrow_pyteal, Mode.Signature, version=8)
print(escrow_teal)

##### Step 3: Compile Teal -> Bytecode for AVM

In [ ]:
# compile Teal -> Bytecode
Escrow = algod_client.compile(escrow_teal)
Escrow

#### Step 4: Deployment – Alice funds the smart signature

In [ ]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(1.5*1e6)
txn = transaction.PaymentTxn(sender=Alice['public'], sp=sp, receiver=Escrow['hash'], amt=amt)

# Step 2+3: sign and sen
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

In [ ]:
# Smart Signature is now funded
print('https://testnet.algoexplorer.io/address/'+ Escrow['hash'])

##### Step 5: Alice informs Bob

In [ ]:
print("Alice communicates to Bob the following")
print("Compiled smart signature:", Escrow['result'])
print("Address of smart signature: ", Escrow['hash'])

#### Step 6: Bob proposes a transaction to the smart signature
* Again he proposes a payment from the cashmachine to **himself**
* The payment transaction is signed by the smart signature, **if the conditions are fullfilled** (correct recipient)

In [ ]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(1.298*1e6) -  int(0.001*1e6)  -  int(0.1*1e6)
txn = PaymentTxn(sender=Escrow['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Escrow['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

### Exercise
* Repeat Step 6. Can Bob withdraw several times?
* Write the transaction(s) for Charlie to try to withdraw from the Escrow

In [ ]:
# Python code goes here